In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
df = pd.read_csv('classification.csv')
df.head()

,Color,Type,Origin,Stolen
0,Red,Sports,Domestic,Yes
1,Red,Sports,Domestic,No
2,Red,Sports,Domestic,Yes
3,Yellow,Sports,Domestic,No
4,Yellow,Sports,Imported,Yes


In [9]:
def gini_feature(df, feature, target):
    values = df[feature].unique()
    feature_count = {str(key): value for key, value in Counter(df[feature]).items()}
    
    count = {}
    for val in values:
        for tar in target:
            count.setdefault(f"{val},{tar}", 0)
    
    for _, row in df.iterrows():
        count[str(row[feature]) + "," + str(row[df.columns[-1]])] += 1
        
    for ele in count:
        count[ele] = (count[ele] / feature_count[ele.split(",")[0]])**2
    final_dict = {}
    for val in values:
        total = 0
        for ele in count:
            if ele.split(",")[0] == val:
                total += count[ele]
        final_dict[val] = 1 - total
    
    for val in final_dict:
        final_dict[val] = final_dict[val] * (feature_count[val]/len(df))
    print(final_dict)
    return sum(final_dict.values())

In [10]:
rootNode = ""
min_val = 1000000
target = list(df[df.columns[-1]].unique())
for feature in df.columns[:-1]:
    curr_val = gini_feature(df, feature, target)
    print(curr_val , " " , min_val)
    if curr_val < min_val:
        min_val = curr_val
        rootNode = feature

print(rootNode)

{'Red': 0.24, 'Yellow': 0.24}
0.48   1000000
{'Sports': 0.26666666666666666, 'SUV': 0.15000000000000002}
0.4166666666666667   0.48
{'Domestic': 0.24, 'Imported': 0.24}
0.48   0.4166666666666667
Type


## DA_6

In [19]:
import pandas as pd
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

df = pd.read_csv('classification.csv')

def gini_feature(df, feature, target_values):
    values = df[feature].unique()
    feature_count = {str(key): value for key, value in Counter(df[feature]).items()}
    count = {}
    for value in values:
        for target_value in target_values:
            count.setdefault(f"{value} {target_value}", 0)
    for _, row in df.iterrows():
        count[str(row[feature]) + " " + row[df.columns[-1]]] += 1
    for ele in count:
        count[ele] = (count[ele] / feature_count[ele.split()[0]])**2
    
    final_dict = {}
    for feature_val in values:
        total = 0
        for ele in count:
            if feature_val == ele.split(" ")[0]:
                total += count[ele]
        final_dict[str(feature_val)] = 1 - total
    
    for val in final_dict:
        final_dict[val] = final_dict[val] * (feature_count[val] / len(df))
    return sum(final_dict.values())

rootNode = ""
min_val = float('inf')
target_values = list(df[df.columns[-1]].unique())
for feature in df.columns[:-1]:
    curr_val = gini_feature(df, feature, target_values)
    if curr_val < min_val:
        rootNode = feature
        min_val = curr_val
print("Selected root feature:", rootNode)

df['predicted_team'] = df[rootNode].apply(lambda x: 'Red Bull' if x == 'Red' else 'Mercedes')
df['actual'] = df['Team'].apply(lambda x: 1 if x == 'Red Bull' else 0)
df['predicted'] = df['predicted_team'].apply(lambda x: 1 if x == 'Red Bull' else 0)

def evaluate_classification(df):
    actual = df['actual']
    predicted = df['predicted']    
    # Accuracy
    accuracy = accuracy_score(actual, predicted)
    # Precision
    precision = precision_score(actual, predicted)
    # Recall
    recall = recall_score(actual, predicted)
    f1 = f1_score(actual, predicted)
    tn, fp, fn, tp = confusion_matrix(actual, predicted).ravel()
    specificity = tn / (tn + fp)
    roc_auc = roc_auc_score(actual, predicted)
    print("Evaluation Metrics:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall (Sensitivity): {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"Specificity: {specificity:.2f}")
    print(f"ROC AUC Score: {roc_auc:.2f}")

evaluate_classification(df)

Selected root feature: Aerodynamics
  Color Engine Type Top Speed Aerodynamics      Team predicted_team
0   Red      Hybrid      Fast         Good  Red Bull       Mercedes
1  Blue    Electric    Medium    Excellent  Mercedes       Mercedes
2  Blue      Hybrid      Fast         Fair  Red Bull       Mercedes
3   Red      Hybrid    Medium         Good  Red Bull       Mercedes
4  Blue    Electric      Fast         Fair  Mercedes       Mercedes
5   Red      Hybrid    Medium    Excellent  Mercedes       Mercedes
6  Blue    Electric      Fast         Good  Red Bull       Mercedes
7   Red      Hybrid      Fast    Excellent  Red Bull       Mercedes
Evaluation Metrics:
Accuracy: 0.38
Precision: 0.00
Recall (Sensitivity): 0.00
F1 Score: 0.00
Specificity: 1.00
ROC AUC Score: 0.50


c:\Users\91984\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
